# Homework 1

#### EE-556 Mathematics of Data - Fall 2024

In this homework, we consider a binary classification task that we will model using logistic regression. Your goal will be to find a classifier using first-order methods and accelerated gradient descent methods. The first part will consist of more theoretical questions, and the second one will ask you to implement these methods.

<div class="alert alert-info">
  ℹ️ <strong>Information on group based work:</strong>
</div>

- You are to deliver only 1 notebook per group.
- Asking assistance beyond your group is ok, but answers should be individual to the group.
- In the event that there was <span style="color: red;">disproportional work done</span> by different group members, let the TAs know.

<div style="border: 1px solid #f00; background-color: #fdd; padding: 10px; border-radius: 5px;">
  ⚠️ Do not forget: Write who are the people in your group as well as their respective SCIPER numbers
</div>

Person 1 **Name**: François Dumoncel || Person 1 **SCIPER**: 314420

Person 2 **Name**:  || Person 2 **SCIPER**:


Person 3 **Name**: || Person 3 **SCIPER**:

## 1. Logistic Regression - 15 Points

Logistic regression is a classic approach to _binary classification_. Before we dive in, let us first define the standard logistic function $\sigma$ on which most of what follows is built:

\begin{equation*}
    \sigma : x \mapsto \frac{1}{1 + \exp{(-x)}}.
\end{equation*}

In logistic regression, we model the _conditional probability_ of observing a class label $b$ given a set of features $\mathbf{a}$. More formally, if we observe $n$ independent samples

\begin{equation*}
    \{(\mathbf{a}_i,b_i)\}_{i=1}^n,
\end{equation*}

where $\mathbf{a}_i\in\mathbb{R}^p$ and $b_i\in\{-1, +1\}$ is the class label, we _assume_ that $b_i$ given $\mathbf{a}_i$ is a symmetric Bernouilli random variable with parameter $\sigma(\mathbf{a}_i^T\mathbf{x}^\natural)$, for some unknown $\mathbf{x}^\natural \in \mathbb{R}^p$. In other words, we assume that there exists an $\mathbf{x}^\natural \in \mathbb{R}^p$ such that

\begin{equation*}
    \mathbb{P}(b_i = 1 \mid \mathbf{a}_i) = \sigma(\mathbf{a}_i^T\mathbf{x}^\natural) \quad \text{ and } \quad \mathbb{P}(b_i = -1 \mid \mathbf{a}_i) = 1 - \sigma(\mathbf{a}_i^T\mathbf{x}^\natural)=  \sigma( - \mathbf{a}_i^T\mathbf{x}^\natural).
\end{equation*}

This is our statistical model. It can be written in a more compact form as follows,

\begin{equation*}
    \mathbb{P}(b_i = j \mid \mathbf{a}_i) = \sigma(j \cdot \mathbf{a}_i^T\mathbf{x}^\natural), \quad j \in \{+1, -1\}.
\end{equation*}

Our goal now is to determine the unknown $\mathbf{x}^\natural$ by constructing an estimator.

__(a)__ (1 point) We are provided with a set of $n$ independent observations. Show that the negative log-likelihood $f$ can be written as:

\begin{equation*}
	\begin{aligned}
		f(\mathbf{x}) = -\log(\mathbb{P}(b_1, \dots, b_n | a_1, \dots, a_n)) & = \sum_{i=1}^n  \log(1 + \exp(- b_i \mathbf{a}_i^T\mathbf{x})).
	\end{aligned}
\end{equation*}


<div style="padding-left: 30px; color:blue">

$
\begin{aligned}
		f(\mathbf{x}) = -\log(\mathbb{P}(b_1, \dots, b_n | a_1, \dots, a_n)) &= -\log\left(\prod_{i=1}^n \mathbb{P}(b_i | \mathbf{a}_i)\right) \\
        &= -\sum_{i=1}^n \log(\sigma(b_i \mathbf{a}_i^\top\mathbf{x})) \\
        &= -\sum_{i=1}^n \log\left((1+\exp(- b_i \mathbf{a}_i^\top\mathbf{x}))^{-1}\right)\\
        &=\sum_{i=1}^n  \log(1 + \exp(- b_i \mathbf{a}_i^T\mathbf{x})).
\end{aligned}
$


</div>

__(b)__ (2 point) Show that the function $u \mapsto \log(1 + \exp(-u))$ is convex. Deduce that $f$ is convex.

<div style="padding-left: 00px; color:blue">

We show that $u \mapsto \log(1 + \exp(-u))$ is convex by showing that it's second derivative is non-negative everywhere

$
\hspace{1cm}
\begin{aligned}
    \frac{\partial}{\partial u}\log(1 + \exp(-u)) &= - \dfrac{e^{-u}}{1+e^{-u}} \\
    \frac{\partial^2}{\partial^2 u}\log(1 + \exp(-u)) &= \dfrac{(1+e^{-u})(-e^{-u})' + e^{-u}(1+e^{-u})'}{(1+e^{-u})^2} \\
    &= \dfrac{e^{-u}(1+e^{-u}) - e^{-2u}}{(1+e^{-u})^2} \\
    &= \dfrac{e^{-u}}{(1+e^{-u})^2} \geq 0, \quad \forall x \in \mathbb{R}. \\
\end{aligned}
$

Therefore, $u \mapsto \log(1 + \exp(-u))$ is convex. Moreover, $f$ is convex because it is the sum of convex functions.

</div>

You have just established that the negative log-likelihood is a convex function. So in principle, any local minimum of the maximum likelihood estimator, which is defined as

\begin{equation*}
    \mathbf{x}^\star_{ML} = \arg\min_{\mathbf{x} \in \mathbb{R}^p} f(\mathbf{x}),
\end{equation*}

is a global minimum so it can serve as an estimator of $\mathbf{x}^\natural$. But, does the minimum always exist? We will ponder this question in the following three points.

__(c)__ (1 point) Explain the difference between infima and minima.  Give an example of a convex function, defined over $\mathbb{R}$, that does not attain its infimum. 

<div style="padding-left: 0px; color:blue">

The minimum of a set $\mathcal{X}$ is the smallest value of the set $\mathcal{X}$. The infimum of a function $f$ is the greatest lower bound of the set of values that $f$ can take. For instance, the function $x  \mapsto e^x$ is defined and convex over $\mathbb{R}$, but it does not attain its infimum which is $0$.

</div>

__(d)__ (1 point) Let us assume that there exists $\mathbf{x}_0 \in \mathbb{R}^p$ such that 

\begin{equation*}
    \forall i\in\{1, \dots, n\}, \quad \quad b_i \mathbf{a}_i^T\mathbf{x}_0 > 0.
\end{equation*}

This is called _complete separation_ in the literature. Can you think of a geometric reason why this name is appropriate? Think of a 2D example where this can happen (i.e $p=2$) and describe why _complete separation_ is an appropriate name.



<html>
  <style>
  .container {
  display: flex;
  align-items: center;
  justify-content: center
}
  </style>
  <body>
    <div class="container">
      <div class="image">
        <img style="padding-left:0px;" src="image.svg" width="350" height="350" />
      </div>
      <div style="color:blue; padding-right:0px">
        <p>As one can see in this toy example, we have two completely separable classes (positive and negative samples). The blue vector defines a separating hyperplane: the black line that completely separates the 2 classes.</p>
      </div>
    </div>
  </body>
</html>

From this, you should see that it is likely that our data satisfies the complete separation assumption. Unfortunately, as you will show in the following question, this can become an obstacle.

__(e)__ (1 point) In a _complete separation_ setting, i.e, there exists $\mathbf{x}_0$ such that 

\begin{equation*}
    \forall i\in\{1, \dots, n\}, \quad \quad b_i \mathbf{a}_i^T\mathbf{x}_0 > 0,
\end{equation*}

prove that the function $f$ does not attain its minimum. 

__Hint__: If the function did have a minimum, would it be above, below or equal to zero? Then think of how $f(2 \mathbf{x}_0)$ compares with $f(\mathbf{x}_0)$, how about $f(\alpha \mathbf{x}_0)$ for $\alpha \rightarrow + \infty$ ? 

<div style="padding-left: 00px; color:blue">

We want to minimize $f(\mathbf{x}) = \sum_{i=1}^{n} \log(1+\exp(-b_i a_i^\top\mathbf{x}))$ over $\mathbb{R}^p$. Let $g(\alpha): \alpha \mapsto f(\alpha \mathbf{x}_0)$, then $g$ decreases and converges to $0$ as $\alpha$ goes to $+\infty$. Therefore, $f$ does not attain its minimum, $0$ is the infimum of $f$.

</div>

As you have just shown convex functions do not always attain their infimum. So it is possible for the maximum-likelihood estimator $\mathbf{x}^\star_{ML}$ to not exist. We will resolve this issue by adding a regularizer. 

In what follows, we consider the function

\begin{equation*}
	f_\mu(\mathbf{x}) = f(\mathbf{x}) + \frac{\mu}{2}\|\mathbf{x}\|_2^2
\end{equation*}
with $\mu> 0$.

__(f)__ (1 point) Show that the gradient of $f_\mu$ can be expressed as 
\begin{equation}
	\nabla f_\mu(\mathbf{x}) = \sum_{i=1}^n -b_i \sigma(-b_i \cdot \mathbf{a}_i^T\mathbf{x})\mathbf{a}_i + \mu \mathbf{x}.
\tag{1}
\end{equation}
__Hint__: Lecture 3 shows you how to proceed with this question.

<div style="color:blue;">

By linearity, we have $\nabla f_\mu(\mathbf{x}) = \nabla f(\mathbf{x}) + \frac{\mu}{2} \nabla \|\mathbf{x}\|_2^2$. 

We know that $\dfrac{\partial}{\partial x_i} (x_1^2 + \dots + x_p^2) = 2x_i, \; \forall i \in [1, p].$ Hence $\nabla \|\mathbf{x}\|_2^2 = 2\mathbf{x}.$

Now for $\nabla f(\mathbf{x})$, we use the following theorem 

$$
\mathcal{J}_{g \circ f} = \mathcal{J}_g(f(\mathbf{x})) \cdot \mathcal{J}_f(\mathbf{x})
$$

The function $\mathbf{x} \mapsto \log(1 + \exp(-\mathbf{a}^\top\mathbf{x}))$ can be rewritten as $g \circ f$ with $g(u) = \log(u)$ and $f(\mathbf{x}) = 1 + \exp(-b_i\mathbf{a}_i^{\top}\mathbf{x})$.

$$
\mathcal{J}_f(\mathbf{x}) = -b_i\mathbf{a}_i\exp(-b_i\mathbf{a}_i^{\top}\mathbf{x}) \quad \text{and} \quad \mathcal{J}_g(u) = \frac{1}{u}
$$
Thus,

$$
\mathcal{J}_g(f(\mathbf{x})) \cdot \mathcal{J}_f(\mathbf{x}) = -\frac{b_i\mathbf{a}_i\exp(-b_i\mathbf{a}_i^{\top}\mathbf{x}) }{1 + \exp(-b_i\mathbf{a}_i^{\top}\mathbf{x})}
$$

Note also that $\sigma(-b_i\mathbf{a}_i^{\top}\mathbf{x}) = \frac{1}{1 + \exp(b_i\mathbf{a}_i^{\top}\mathbf{x})} = \frac{\exp(-b_i\mathbf{a}_i^{\top}\mathbf{x})}{1 + \exp(-b_i\mathbf{a}_i^{\top}\mathbf{x})}$ .

We can now conclude the proof by summing over all point

$$
\begin{aligned}
\nabla f_\mu(\mathbf{x}) &= \sum_{i=1}^n \nabla\log(1 + \exp(-b_i\mathbf{a}_i^{\top}\mathbf{x})) + \mu \mathbf{x} \\
&= \sum_{i=1}^n -b_i \sigma(-b_i \cdot \mathbf{a}_i^T\mathbf{x})\mathbf{a}_i + \mu \mathbf{x}.
\end{aligned}
$$

</div>

__(g)__ (1 point) Show that the Hessian of $f_\mu$ can be expressed as 
\begin{equation}
	\nabla^2 f_\mu(\mathbf{x}) = \sum_{i=1}^{n} \sigma(-b_i \cdot \mathbf{a}_i^T\mathbf{x})(1 - \sigma(-b_i \cdot \mathbf{a}_i^T\mathbf{x}))\mathbf{a}_i\mathbf{a}_i^T + \mu \mathbf{I}.
\tag{2}
\end{equation}

It is convenient to observe that we can write the Hessian in a more compact form by defining the matrix
\begin{equation}
	\mathbf{A} = \begin{bmatrix}
        \leftarrow &  \mathbf{a}_1^T & \rightarrow \\
        \leftarrow &  \mathbf{a}_2^T & \rightarrow \\
         &  \ldots &  \\
        \leftarrow &  \mathbf{a}_n^T & \rightarrow \\
  \end{bmatrix}.
\end{equation}
It is easy to see that we have
\begin{equation}
	\nabla^2 f_\mu(\mathbf{x}) =  \mathbf{A}^T \text{Diag}\left(\sigma(-b_i \cdot \mathbf{a}_i^T\mathbf{x})(1 - \sigma(-b_i \cdot \mathbf{a}_i^T\mathbf{x})) \right)\mathbf{A}+ \mu \mathbf{I}.
\end{equation}


__(h)__ (1 point) Show that $f_\mu$ is $\mu$-strongly convex. 

__(i)__ (3 points) Is it possible for a strongly convex function to not attain its minimum ? <a name="cite_ref-1"></a>[<sup>[1]</sup>](#cite_note-1) Justify your reasoning.

We will now show that $f_\mu$ is smooth, i.e, $\nabla f_\mu$ is L-Lipschitz with respect to the Euclidean norm, with 
\begin{equation}
	L = \|A\|^2_F + \mu \text{, where }\|\cdot\|_F\text{ denotes the Frobenius norm. }
\end{equation}

(1 point for all three questions)

__(j-1)__ Show that $\lambda_{\max}(\mathbf{a}_i\mathbf{a}_i^T) = \left\| \mathbf{a}_i\right\|_2^2$, where $\lambda_{\max}(\cdot)$ denotes the largest eigenvalue.

__(j-2)__ Using [2](#mjx-eqn-eq2), show that $\lambda_{\max}(\nabla^2 f_\mu(\mathbf{x})) \leq \sum_{i=1}^{n} \|\mathbf{a}_i\|_2^2 + \mu$. 

__Hint__: Recall that $\lambda_{\max}(\cdot)$ verifies the triangle inequality.

__(j-3)__ Conclude that $f_\mu$ is $L$-smooth for $L = \|A\|_F^2 + \mu$.

__(l)__ (2 point) To finalize, we introduce the Kullback-Leibler (KL) divergence. The KL divergence is a measure of how one probability distribution differs from a second, reference probability distribution. Along side the examples we saw in slide 18 of Lecture 1, the KL divergence is also a useful loss function to be used in learning frameworks.

Write the definition of the Kullback-Leibler (KL) divergence between the true label distribution $q(b_i)$ and the model’s predicted distribution $p(b_i∣\mathbf{a}_i)$ and show that minimizing the KL divergence between $q(b_i)$ and $p(b_i∣\mathbf{a}_i)$ is equivalent to minimizing the negative log-likelihood derived in (a).


From your work in this section, you have shown that the maximum likelihood estimator for logistic regression might not exist, but it can be guaranteed to exist by adding a $\|\cdot\|_2^2$ regularizer. Consequently, the estimator for $\mathbf{x}^\natural$ we will use will be the solution of the smooth strongly convex problem,
\begin{equation}
	\mathbf{x}^\star=\arg\min_{\mathbf{x} \in \mathbb{R}^p} f(\mathbf{x}) + \frac{\mu}{2}\|\mathbf{x}\|_2^2.
\tag{3}
\end{equation}

<a name="cite_note-1"></a>1. [^](#cite_ref-1) TA's will give you candy if you provide a complete proof.